In [1]:
library(selbal)
library(reshape)
library(tidyr)


Attaching package: 'tidyr'

The following objects are masked from 'package:reshape':

    expand, smiths



In [2]:
run_selbal <- function(dataset_name, method, rank) {
    #metaphlan2/VogtmannE_2016.csv"

    path <- "/gfs/data/curated_metagenomes_"
    path = paste(path, method, sep="")
    path = paste(path, dataset_name, sep="/")
    path = paste(path, "csv", sep=".")

    
    print('Reading file')
    mydata <- read.csv(path, header = TRUE, strip.white=TRUE)
    df <- data.frame(mydata)
    print('df ready')

    df_g <- df[df$Rank==rank,]
    df_g <- df_g[df_g$STUDY_CONDITION != '',]
    print('df_g ready.')
    df_g_subset <- subset(df_g, select=c("Sample_ID", "Name", "Abundance"))
    
    print('pivot started....')

    pivot <- spread(df_g_subset, Name, Abundance)
    
    #pivot <- cast(df_g_subset, Sample_ID ~ Name)
    print('Pivot ready.')

    df_cond <- unique(subset(df_g, select=c("Sample_ID", "STUDY_CONDITION")))

    merged <- merge(x = pivot, y = df_cond, by = "Sample_ID", )
    print('Merge ready.')

    merged_sel <- merged[merged$STUDY_CONDITION!='adenoma',]
    head(merged_sel)

    cols = dim(merged_sel)[2]

    # Define x, y and z
    x <- merged_sel[,3:cols-1]
    print('x ready.')
    #y <- merged_sel[,cols]

    #merged_sel$CRC <- merged_sel$STUDY_CONDITION == "CRC"
    #merged_sel$CRC <- replace(merged_sel$CRC, merged_sel$CRC, "Pos")
    #merged_sel$CRC <- replace(merged_sel$CRC, merged_sel$CRC != "Pos", "Neg")
    #y <- merged_sel$CRC
    #y_fact <- factor(c(y))

    #y_fact <- factor(c(merged_sel$STUDY_CONDITION))
    y_fact <- factor(merged_sel$STUDY_CONDITION)
    print('y ready.')
    #merged_sel$STUDY_CONDITION[merged_sel$STUDY_CONDITION != 1]  <- 0

    #min(merged_sel[!is.na(merged_sel)])
    #b = apply(merged_sel, 1, function(x) 1/as.numeric(min(x, na.rm = TRUE)))
    x_2 = x

    #for (r in 1:dim(x_2)[1]){
    #    x_2[r,] = round(x_2[r,]*b[r])
    #}

    x_2[is.na(x_2)] <- 0
    
    print('x2 un-zerod.')

    rows = dim(x_2)[1]

    for (col_name in colnames(x_2)){
    col = x_2[col_name]
    if (sum(col != 0) <= rows*0.2){
      drop <- c(col_name)
      x_2 = x_2[,!(names(x_2) %in% drop)]
    }
    }
    print('x2 ready.')
    x_2[is.na(x_2)] <- 0
    
    print('Selbal started.')
    
    
    result <- selbal.cv(x = x_2, y = y_fact, covar = NULL, n.fold = 5, n.iter = 10, logit.acc = "AUC")
    print('Selbal finished.')
    
    path_for_plot = "save/"
    path_for_plot = paste(path_for_plot, dataset_name, sep="")
    path_for_plot = paste(path_for_plot, method, sep="_")
    path_for_plot = paste(path_for_plot, rank, sep="_")

    svg(paste(path_for_plot, "accuracy.svg", sep="_"))
    plot(result$accuracy.nvar)
    dev.off()

    svg(paste(path_for_plot, "barplot.svg", sep="_"))
    plot(result$var.barplot)
    dev.off()

    svg(paste(path_for_plot, "globalplot.svg", sep="_"))
    grid.draw(result$global.plot)
    dev.off()

    svg(paste(path_for_plot, "cvtab.svg", sep="_"))
    plot.tab(result$cv.tab)
    dev.off()

    return (result)
}

In [3]:
result = run_selbal("FengQ_2015", "metaphlan2", "p")

[1] "Reading file"
[1] "df ready"
[1] "df_g ready."
[1] "pivot started...."
[1] "Pivot ready."
[1] "Merge ready."
[1] "x ready."
[1] "y ready."
[1] "x2 un-zerod."
[1] "x2 ready."
[1] "Selbal started."


############################################################### 
 STARTING selbal.cv FUNCTION 
###############################################################

#-------------------------------------------------------------# 
# ZERO REPLACEMENT . . .



Loading required package: MASS
Loading required package: NADA
Loading required package: survival

Attaching package: 'NADA'

The following object is masked from 'package:stats':

    cor

Loading required package: truncnorm



, . . . FINISHED. 
#-------------------------------------------------------------#

#-------------------------------------------------------------# 
# Starting the cross - validation procedure . . .
 . . . finished. 
#-------------------------------------------------------------# 
###############################################################

 The optimal number of variables is: 3 



Setting levels: control = CRC, case = control
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases

Attaching package: 'gridExtra'

The following object is masked from 'package:Biobase':

    combine

The following object is masked from 'package:BiocGenerics':

    combine





############################################################### 
 . . . FINISHED. 
###############################################################[1] "Selbal finished."


Warning message in svg(paste(path_for_plot, "accuracy.svg", sep = "_")):
"cairo error 'error while writing to output stream'"

ERROR: Error in svg(paste(path_for_plot, "accuracy.svg", sep = "_")): unable to start device 'svg'
